In [2]:
import cml.data_v1 as cmldata

CONNECTION_NAME = "go01-aw-dl"
conn = cmldata.get_connection(CONNECTION_NAME)
spark = conn.get_spark_session()

# Sample usage to run query through spark
EXAMPLE_SQL_QUERY = "show databases"
spark.sql(EXAMPLE_SQL_QUERY).show()


DEBUG: Starting new HTTPS connection (1): ml-4c5feac0-3ec.go01-dem.ylcu-atmi.cloudera.site:443
DEBUG: https://ml-4c5feac0-3ec.go01-dem.ylcu-atmi.cloudera.site:443 "GET /api/v1/projects/1888/data-connections HTTP/1.1" 200 None
DEBUG: https://ml-4c5feac0-3ec.go01-dem.ylcu-atmi.cloudera.site:443 "GET /api/v1/projects/1888/data-connections/33176?engine_id=0q1uzvvu06j5t9iq HTTP/1.1" 200 785
Setting spark.hadoop.yarn.resourcemanager.principal to pauldefusco
Hive Session ID = 24fc3e18-d270-4f2e-810f-bf43cf78de56


+--------------------+
|           namespace|
+--------------------+
|         01_car_data|
|           01_car_dw|
|      adash_car_data|
|             airline|
|          airline_dw|
|            airlines|
|        airlines_csv|
|       airlines_csv1|
|   airlines_csv_vish|
|    airlines_iceberg|
|   airlines_iceberg1|
|airlines_iceberg_...|
|airlines_iceberg_...|
|      airlines_mjain|
|          airquality|
|          atlas_demo|
|            bankdemo|
|          bca_jps_l0|
|          bca_jps_l1|
|              bhagan|
+--------------------+
only showing top 20 rows



In [5]:
import dbldatagen as dg

dfSource = spark.read.format("csv").load("data/car_sales.csv")

In [9]:
analyzer = dg.DataAnalyzer(sparkSession=spark, df=dfSource)

In [10]:
code =  dg.DataAnalyzer.scriptDataGeneratorFromSchema(dfSource.schema)

AttributeError: type object 'DataAnalyzer' has no attribute 'scriptDataGeneratorFromSchema'

In [21]:
s3BucketName = "data"

In [22]:
car_installs  = spark.read.csv(s3BucketName + "/car_installs.csv",        header=True, inferSchema=True)
car_sales     = spark.read.csv(s3BucketName + "/historical_car_sales.csv",           header=True, inferSchema=True)
customer_data = spark.read.csv(s3BucketName + "/customer_data.csv",       header=True, inferSchema=True)
factory_data  = spark.read.csv(s3BucketName + "/experimental_motors.csv", header=True, inferSchema=True)
geo_data      = spark.read.csv(s3BucketName + "/postal_codes.csv",        header=True, inferSchema=True)

In [18]:
partitions_num=10 
row_count = 100000 
unique_vals=100000 
display_option=True

In [24]:
print("car installs")
print(car_installs.schema)
print("\n car sales")
print(car_sales.schema)
print("\n customer_data")
print(customer_data.schema)
print("\n factory_data")
print(factory_data.schema)
print("\n geo_data")
print(geo_data.schema)

car installs
StructType(List(StructField(model,StringType,true),StructField(VIN,StringType,true),StructField(serial_no,StringType,true)))

 car sales
StructType(List(StructField(customer_id,IntegerType,true),StructField(model,StringType,true),StructField(saleprice,DoubleType,true),StructField(VIN,StringType,true),StructField(month,IntegerType,true),StructField(year,IntegerType,true),StructField(day,IntegerType,true)))

 customer_data
StructType(List(StructField(customer_id,IntegerType,true),StructField(username,StringType,true),StructField(name,StringType,true),StructField(gender,StringType,true),StructField(email,StringType,true),StructField(occupation,StringType,true),StructField(birthdate,StringType,true),StructField(address,StringType,true),StructField(salary,DoubleType,true),StructField(zip,IntegerType,true)))

 factory_data
StructType(List(StructField(factory_no,IntegerType,true),StructField(machine_no,IntegerType,true),StructField(part_no,StringType,true),StructField(serial_no,S

In [45]:
customer_data.take(1)

[Row(customer_id=1, username='robin48', name='Jesse Spencer', gender='M', email='udalton@yahoo.com', occupation='Pharmacist, community', birthdate='1975-09-24', address='10305 Scott River Suite 976New Rachelside, SD 17002', salary=153032.45, zip=5767)]

In [52]:
factory_data.select('status').distinct().show()

+-----------+
|     status|
+-----------+
|beta_engine|
+-----------+



In [66]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
from pyspark.sql.types import LongType, IntegerType, StringType
import dbldatagen as dg

In [58]:
def car_installs_gen(partitions_num=10, row_count = 100000, unique_vals=100000, display_option=True):
    
    model_codes = ["A","B","D","E"]

    testDataSpec = (
        dg.DataGenerator(spark, name="car_installs", rows=row_count,partitions=partitions_num).withIdOutput()
        .withColumn("model", "string", values=model_codes, random=True)
        .withColumn("VIN", "string", template=r'\\N8UCGTTVDK5J', random=True)
        .withColumn("serial_no", "string", template=r'\\N42CLDR0156661577860220', random=True)
    )

    dfTestData = testDataSpec.build()
    
    return dfTestData

In [59]:
def car_sales_gen(partitions_num=10, row_count = 100000, unique_vals=100000, display_option=True):
    
    model_codes = ["Model A","Model B","Model D","Model E"]

    testDataSpec = (
        dg.DataGenerator(spark, name="car_sales", rows=row_count,partitions=partitions_num).withIdOutput()
        .withColumn("customer_id", "integer", minValue=10000, maxValue=1000000, random=True)
        .withColumn("model", "string", values=model_codes, random=True)
        .withColumn("saleprice", "decimal(10,2)", minValue=5000, maxValue=100000, random=True)
        .withColumn("VIN", "string", template=r'\\N8UCGTTVDK5J', random=True)
        .withColumn("month", "integer", minValue=1, maxValue=12, random=True)
        .withColumn("year", "integer", minValue=1999, maxValue=2023, random=True)
        .withColumn("day", "integer", minValue=1, maxValue=28, random=True)
    )

    dfTestData = testDataSpec.build()
    
    return dfTestData

In [60]:
def customer_gen(partitions_num=10, row_count = 100000, unique_vals=100000, display_option=True):
    
    model_codes = ["Model A","Model B","Model D","Model E"]
    gender_codes = ["M","F"]

    testDataSpec = (
        dg.DataGenerator(spark, name="customer_data", rows=row_count,partitions=partitions_num).withIdOutput()
        .withColumn("customer_id", "integer", minValue=10000, maxValue=1000000, random=True)
        .withColumn('username', 'string', template=r'\\w', random=True)
        .withColumn('name', 'string', template=r'\\w', random=True)
        .withColumn('gender', 'string', values=gender_codes, random=True)
        .withColumn("email", 'string', template=r"\\w.\\w@\\w.com", random=True)
        .withColumn("birthdate", "timestamp", begin="1950-01-01 01:00:00", 
                end="2003-12-31 23:59:00", interval="1 minute", random=True )
        .withColumn("salary", "decimal(10,2)", minValue=50000, maxValue=1000000, random=True)
        .withColumn("zip", "integer", minValue=10000, maxValue=99999, random=True)
    )

    dfTestData = testDataSpec.build()
    
    return dfTestData

In [63]:
def factory_gen(partitions_num=10, row_count = 100000, unique_vals=100000, display_option=True):

    testDataSpec = (
        dg.DataGenerator(spark, name="factory_data", rows=row_count,partitions=partitions_num).withIdOutput()
        .withColumn("factory_no", "int", minValue=10000, maxValue=1000000, random=True)
        .withColumn("machine_no", "int", minValue=120, maxValue=99999, random=True)
        .withColumn("serial_no", "string", template=r'\\N42CLDR0156661577860220', random=True)
        .withColumn("part_no", "string", template=r'\\a42CLDR', random=True)
        .withColumn("timestamp", "timestamp", begin="2000-01-01 01:00:00", 
                end="2003-12-31 23:59:00", interval="1 minute", random=True )
        .withColumn("status", "string", values=["beta_engine"])
        
    )

    dfTestData = testDataSpec.build()
    
    return dfTestData

In [62]:
def geo_gen(partitions_num=10, row_count = 100000, unique_vals=100000, display_option=True):

    state_names = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "Florida"]
    
    testDataSpec = (
        dg.DataGenerator(spark, name="geo_data", rows=row_count,partitions=partitions_num).withIdOutput()
        .withColumn("country_code", "string", values=["US"])
        .withColumn("state", "string", values=state_names, random=True)
        .withColumn("postalcode", "integer", minValue=10000, maxValue=99999, random=True)
        .withColumn("latitude", "decimal(10,2)", minValue=-90, maxValue=90, random=True)
        .withColumn("longitude", "decimal(10,2)", minValue=-180, maxValue=180, random=True)
    )

    dfTestData = testDataSpec.build()
    
    return dfTestData

In [64]:
dfTestData = geo_gen()

In [65]:
dfTestData.take(1)

[Row(id=0, country_code='US', state='California', postalcode=18287, latitude=Decimal('52.00'), longitude=Decimal('-15.00'))]

In [19]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
from pyspark.sql.types import LongType, IntegerType, StringType
import dbldatagen as dg

In [20]:
country_codes = [
    "CN", "US", "FR", "CA", "IN", "JM", "IE", "PK", "GB", "IL", "AU",
    "SG", "ES", "GE", "MX", "ET", "SA", "LB", "NL", "IT"
]
country_weights = [
    1300, 365, 67, 38, 1300, 3, 7, 212, 67, 9, 25, 6, 47, 83,
    126, 109, 58, 8, 17, 20
]

manufacturers = [
    "Delta corp", "Xyzzy Inc.", "Lakehouse Ltd", "Acme Corp", "Embanks Devices",
]

lines = ["delta", "xyzzy", "lakehouse", "gadget", "droid"]

testDataSpec = (
    dg.DataGenerator(spark, name="device_data_set", rows=row_count,partitions=partitions_num).withIdOutput()
    # we'll use hash of the base field to generate the ids to
    # avoid a simple incrementing sequence
    .withColumn("internal_device_id", "long", minValue=0x1000000000000,
                uniqueValues=unique_vals, omit=True, baseColumnType="hash",
    )
    # note for format strings, we must use "%lx" not "%x" as the
    # underlying value is a long
    .withColumn(
        "device_id", "string", format="0x%013x", baseColumn="internal_device_id"
    )
    # the device / user attributes will be the same for the same device id
    # so lets use the internal device id as the base column for these attribute
    .withColumn("country", "string", values=country_codes, #weights=country_weights,
                baseColumn="internal_device_id")
    .withColumn("manufacturer", "string", values=manufacturers,
                baseColumn="internal_device_id", )
    # use omit = True if you don't want a column to appear in the final output
    # but just want to use it as part of generation of another column
    .withColumn("line", "string", values=lines, baseColumn="manufacturer",
                baseColumnType="hash", omit=True )
    .withColumn("model_ser", "integer", minValue=1, maxValue=11, baseColumn="device_id",
                baseColumnType="hash", omit=True, )
    .withColumn("model_line", "string", expr="concat(line, '#', model_ser)",
                baseColumn=["line", "model_ser"] )
    .withColumn("event_type", "string",
                values=["activation", "deactivation", "plan change", "telecoms activity",
                        "internet activity", "device error", ],
                random=True)
    .withColumn("event_ts", "timestamp", begin="2020-01-01 01:00:00",
                end="2020-12-31 23:59:00",
                interval="1 minute", random=True )
)

dfTestData = testDataSpec.build()